# RNA-Seq Pipeline: Step 5 - DGE Analysis (DESeq2)

This is the final step: **Differential Gene Expression (DGE)** analysis.

**Goal:**
To take the **counts matrix** (from step 4) and the **sample sheet** (metadata) and perform statistical analysis to find out *which genes* are significantly **up-regulated** (more expressed) or **down-regulated** (less expressed) in the "Treated" group compared to the "Control" group.

**Workflow:**
1.  **Setup:** Load the `R` libraries (DESeq2, tidyverse).
2.  **Load Data:** Read the `gene_counts_matrix.txt` and `sample_sheet.csv` files.
3.  **Clean Data:** Prepare the tables for analysis (e.g., make sure sample names match).
4.  **Run DESeq2:** Run the main DGE statistical model.
5.  **Get Results:** Extract the final results table (Log2FoldChange, p-value).
6.  **Visualize:** Create plots (like a Volcano plot and PCA plot) to visualize the results.

**Tool:** `DESeq2` (an `R` package)

In [ ]:
# --- 1. Load Libraries ---
# 'suppressPackageStartupMessages' just hides the messy startup text
suppressPackageStartupMessages(library(DESeq2))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(EnhancedVolcano))

print("Libraries loaded successfully.")

In [ ]:
# --- 2. Load Data ---
counts_file <- "04_Counts/gene_counts_matrix.txt"
metadata_file <- "docs/sample_sheet.csv"

sample_sheet <- read_csv(metadata_file) %>% 
                  column_to_rownames("SampleID")

counts_data <- read.delim(counts_file, skip = 1, row.names = 1)

# --- 3. Clean Data (THIS IS THE FIX) ---

# (Verification) Let's see the ugly names R loaded:
print("--- Original (Ugly) Column Names from featureCounts ---")
print(colnames(counts_data))

# FIX: Clean the column names.
# We remove the prefix (like "X03_Aligned_BAMs.")
# and the suffix (like ".sorted.bam")

# Create a new, clean list of names
clean_names <- colnames(counts_data)
# Remove the path prefix (handles "X03_..." or "03_...")
clean_names <- gsub("^X?03_Aligned_BAMs\\.", "", clean_names) 
# Remove the file suffix
clean_names <- gsub("\\.sorted\\.bam$", "", clean_names)       

# Assign these new, clean names back to the data frame
colnames(counts_data) <- clean_names

# (Verification) Let's see the new, clean names
print("--- New (Cleaned) Column Names ---")
print(colnames(counts_data))


# Now, select ONLY the 6 columns that match our sample sheet
# (This ignores "Chr", "Start", "End", etc.)
counts_data_cleaned <- counts_data %>% 
                         select(rownames(sample_sheet))

# (Verification) Check if the orders match. This must be TRUE.
print("--- Do columns and rows match? ---")
print(all(colnames(counts_data_cleaned) == rownames(sample_sheet)))


# --- 4. Print Data (to verify) ---
print("--- Sample Sheet (Metadata) ---")
print(sample_sheet)
print("--- Counts Data (Cleaned) ---")
print(head(counts_data_cleaned))

### 4. Run the DESeq2 Analysis

Now that our `counts_data_cleaned` and `sample_sheet` are loaded and aligned, we can run the analysis.

**Workflow:**
1.  **Create DDS Object:** Combine the counts and metadata into a single object called a `DESeqDataSet` (we'll call it `dds`).
2.  **Filter (Optional):** Remove genes with very low counts (e.g., less than 10 reads total) to improve statistical power.
3.  **Run `DESeq()`:** This is the main function. It normalizes the data and fits the statistical model to find differences.

In [ ]:
# --- 5. Run DESeq2 ---

# 1. Create the DESeqDataSet (dds) object
# We tell it our "design" (what we want to test) is the "Condition" column
dds <- DESeqDataSetFromMatrix(countData = counts_data_cleaned,
                              colData = sample_sheet,
                              design = ~ Condition)

# 2. (Optional but recommended) Pre-filter low-count genes
# We keep only genes that have at least 10 reads total across all samples
keep <- rowSums(counts(dds)) >= 10
dds <- dds[keep,]

# 3. Run the main analysis
# This step does normalization, dispersion estimation, and statistical testing
print("--- Running DESeq2 analysis... ---")
dds <- DESeq(dds)
print("--- DESeq2 analysis complete. ---")

# 4. Get the final results table
# We specify the comparison: "Treated" vs "Control"
res <- results(dds, contrast=c("Condition", "Treated", "Control"))

# 5. Print a summary of the results
print("--- Summary of results ---")
print(summary(res))

### 5. Visualize the Results

The statistical analysis is complete. The results are stored in the `res` object.

Now we will do two things:
1.  **Save Results:** Save the full results table to a `.csv` file for our records.
2.  **PCA Plot:** Create a "Principal Component Analysis" plot. This plot checks if our samples cluster together correctly (i.e., all "Control" samples look similar, and all "Treated" samples look similar).
3.  **Volcano Plot:** Create a "Volcano Plot". This is the most famous plot in DGE. It shows all genes at once:
    * **X-axis:** Log2 Fold Change (how much it changed; right = up, left = down).
    * **Y-axis:** P-value (how significant the change is; higher = more significant).

In [ ]:
# --- 6. Save the Results to a File ---

# Create the output directory
output_dir <- "05_DGE_Results"
dir.create(output_dir, showWarnings = FALSE)

# Convert the results to a data.frame and add gene names as a column
res_df <- as.data.frame(res) %>%
          rownames_to_column("GeneID")

# Define the output file path
output_file <- file.path(output_dir, "dge_results_Treated_vs_Control.csv")

# Save the file
write_csv(res_df, output_file)

print(paste("Results saved to:", output_file))
print("--- First 6 rows of the results table: ---")
print(head(res_df))

In [ ]:
# --- 7. Visualization 1: PCA Plot (Sample Quality Control) ---

# Transform the counts data (VST)
vsd <- vst(dds, blind=FALSE)

# Generate the PCA plot and save it to a variable 'p'
p_pca <- plotPCA(vsd, intgroup=c("Condition"))

# --- THIS IS THE NEW LINE ---
# Save the plot 'p' to a file in our results directory
ggsave(file.path(output_dir, "PCA_plot_samples.png"), plot = p_pca)

print("--- PCA Plot (checks sample similarity) ---")
print("Saved plot to 05_DGE_Results/PCA_plot_samples.png")
p_pca # This line prints the plot to the notebook

In [ ]:
# --- 8. Visualization 2: Volcano Plot (Gene Results) ---

print("--- Volcano Plot (shows up/down regulated genes) ---")

# Generate the plot and save it to a variable 'p_volcano'
p_volcano <- EnhancedVolcano(res,
    lab = rownames(res),
    x = 'log2FoldChange',
    y = 'padj', # Use the *adjusted* p-value
    title = 'Treated vs Control',
    pCutoff = 0.05,    # Adjusted p-value cutoff
    FCcutoff = 1.0,    # Log2FoldChange cutoff
    pointSize = 2.0,
    labSize = 3.0)

# --- THIS IS THE UPDATE ---
# Save the volcano plot 'p_volcano' to a file
ggsave(file.path(output_dir, "Volcano_plot_genes.png"), plot = p_volcano, width = 10, height = 10, units = "in")

print("Saved plot to 05_DGE_Results/Volcano_plot_genes.png")
p_volcano # This line prints the plot to the notebook

### 6. Visualization 3: Heatmap of Top Genes

Finally, let's visualize the expression patterns of the most significant genes.

A heatmap allows us to see how the expression of individual genes (rows) changes across all our samples (columns). We will plot the **Top 50** most significant genes (lowest adjusted p-value).

-   We expect to see two main blocks of genes (one up-regulated, one down-regulated).
-   We also expect the samples (columns) to cluster perfectly into "Control" and "Treated" groups, confirming our PCA plot.

In [ ]:
# --- 9. Visualization 3: Heatmap of Top Genes ---

# Load the heatmap library
library(pheatmap)

# 1. Get the top 50 most significant genes (same as before)
top_genes <- head(order(res$padj), 50)

# 2. Get the normalized counts matrix (same as before)
norm_counts_matrix <- assay(vsd)

# 3. Filter the matrix (same as before)
top_genes_matrix <- norm_counts_matrix[top_genes, ]

# 4. Scale the data (same as before)
scaled_matrix <- t(scale(t(top_genes_matrix)))

# --- THIS IS THE UPDATE ---
# 5a. Save the heatmap to a PNG file
png(file.path(output_dir, "Heatmap_Top50_genes.png"), width = 8, height = 7, units = "in", res = 300)
pheatmap(scaled_matrix, 
         annotation_col = sample_sheet,
         show_rownames = FALSE, # Hide gene names
         cluster_rows = TRUE,   # Group similar genes
         cluster_cols = TRUE)  # Group similar samples
dev.off() # This command closes the file

# 5b. Print the heatmap to the notebook (so we can see it)
print("--- Heatmap of Top 50 Significant Genes ---")
print("Saved plot to 05_DGE_Results/Heatmap_Top50_genes.png")
pheatmap(scaled_matrix, 
         annotation_col = sample_sheet,
         show_rownames = FALSE,
         cluster_rows = TRUE,
         cluster_cols = TRUE)